# PANDAS AVANÇADO
---

# CARREGANDO OS DADOS
---

## Configurações do projeto

### Importando pacotes

In [ ]:
!pip install gcsfs 
!pip install pyspark

In [ ]:
#import Bibliotecas
import pandas as pd
import numpy as np
import os

In [ ]:
from pyspark.sql import SparkSession
from pymongo import MongoClient
from google.cloud import storage

###Conexões das bibliotecas

In [ ]:
#Conexão com o drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Conecção com a GCP
serviceAccount = '/content/drive/MyDrive/BC26-ENG DADOS-PYTHON/PROJETOS/PROJETO ESTUDO/Cópia de key-store-via-colab.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
#conector do Mongo Atlas
uri = "mongodb+srv://nayyarabernardo-soulcod.mv7bzjn.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri, 
                     tls=True, 
                     tlsCertificateKeyFile='/content/drive/MyDrive/BC26-ENG DADOS-PYTHON/PROJETOS/PROJETO ESTUDO/Cópia de X509-cert-3376380089754927980.pem')


In [ ]:
#CONFIGURAR A SPARK SESSION
spark = (
   SparkSession.builder
               .master('local')
               .appName('gcsfs') 
               .config('spark.ui.port', '4050')
               .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar') #config acesso conector gcp
               .getOrCreate()                
)

### Opções de configuração

In [ ]:
#Para mostrar todas as colunas no display
pd.set_option('display.max_columns',25)

In [ ]:
pd.get_option('display.max_rows', 1000)

60

## Carregando os dados

### Arquivos JSON - `read_json`

In [ ]:
dados = pd.read_json(
    path_or_buf='gs://projeto_cat/ORIGINAL/realestates.json',
    orient='columns'
)

In [ ]:
dados

,normal,highlights
output,{'listings': [{'imovel': {'tipos': {'proprieda...,{'listings': [{'imovel': {'tipos': {'proprieda...


### Arquivos EXCEL - `read_excel`

In [ ]:
bairros = pd.read_excel(
    io = 'gs://projeto_cat/ORIGINAL/bairros.xlsx',
    sheet_name='Preço médio por tipo',
    usecols="C:E",
    header = 2,
    index_col=[0, 1],
    names=['bairros','tipo','valor_m2_bairro']
)
bairros

## Carregando os dados BRUTOS para o mongo DB

In [ ]:
#Criar conexão
db = client['IMOVEIS']

In [ ]:
# #Criar e enviar coleção ORIGINAL  
colecao_1 = db['realestates-ORIGINAL']
df1_dict = dados.to_dict('records')
# Inserindo os dados no mongodb
colecao_1.insert_many(df1_dict)

In [ ]:
colecao_2_parte1 = db['bairros-ORIGINAL']
df2_dict = bairros.to_dict('records')
colecao_2_parte1.insert_many(df2_dict)

# TRANSFORMANDO E TRATANDO OS DADOS
---

## Transformando dados no formato JSON para uma tabela
- Utilizando o json_normalize para informações em colunas 
- existem duas colunas inicialmente nesse dataframe Coluna Normal e Highlights

In [ ]:
#Transformar essas informações em linhas da minha tabela , ele vai utilizar as informações conditas para nomes das colunas 
dados_normal_listings = pd.json_normalize(data=dados.normal, sep='_', record_path=['listings'])

In [ ]:
dados_highlights_listings = pd.json_normalize(data=dados.highlights, sep='_', record_path=['listings'])
dados_highlights_listings

## Trabalhando com dados textuais
- Foi ferificado que em algumas colunas que existem mais de uma informações aparentimente seriam lista mas ao verificar utlizando o type foi verificado que se tratavam de str e foi feito esse tratamento

### Aanalise das colunas com *strings* em listas do Python

In [ ]:
dados_normal_listings

In [ ]:
lista_str = dados_normal_listings.loc[0, 'imovel_caracteristicas_propriedade']

In [ ]:
type(lista_str)

str

#### Métodos de *strings*

[Documentação](https://docs.python.org/3.6/library/stdtypes.html#string-methods)

In [ ]:
#1° passo eliminar os '[', 2° passo depois replace as aspas por nada e 3° passo retirar as virgulas adicionando cada utem a uma lista 
lista = lista_str[1:-1].replace("'","").split(", ")

#### `str`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html)

#### `filter`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.filter.html)

# COMBINANDO CONJUNTOS DE DADOS
---

## Empilhando *DataFrames*

### `append`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html)

### `concat`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)

## Combinando *DataFrames* a partir de uma chave de ligação

### Tratando a variável de ligação

#### Selecionando a variável de ligação nos dois *DataFrames*

[Documentação: `get_level_values`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.get_level_values.html)

#### Identificando e eliminando valores duplicados - `nunique` e `unique`

[Documentação: `nunique`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.nunique.html)

[Documentação: `unique`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unique.html)

---

#### Verificando se existe correspondência entre as variáveis de ligação

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.isin.html)

#### Ajustando os valores

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.replace.html)

### Combinando os *DataFrames*

#### `merge`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html)

```
pandas.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes='_x', '_y', copy=True, indicator=False, validate=None)
```

**Parameters**

- **left**: DataFrame


- **right**: DataFrame
    

- **how**: {‘left’, ‘right’, ‘outer’, ‘inner’}, *default* ‘inner’

        Tipo de junção que será realizada.

        - left: usa apenas as chaves do DataFrame especificado no parâmetro left.

        - right: usa apenas as chaves do DataFrame especificado no parâmetro right.

        - outer: considera todos os registros dos DataFrames especificados nos parâmetros left e right, isto é, faz a união das chaves.

        - inner: considera apenas a interseção das chaves dos DataFrames especificados nos parâmetros left e right.
    

- **on**: label ou list

        Nomes das colunas ou dos índices que serão utilizados na junção. Estes devem ser encontrados em ambos os DataFrames.
        

- **left_on**: label ou list, or array-like

        Nomes das colunas ou dos índices do DataFrame especificado em left que serão utilizados na junção.
        

- **right_on**: label or list, or array-like

        Nomes das colunas ou dos índices do DataFrame especificado em right que serão utilizados na junção.
    

- **left_index**: bool, *default False*

        Indica se o índice do DataFrame especificado no parâmetro left deve ser utilizado como chave de junção.
        

- **right_index**: bool, *default False*

        Indica se o índice do DataFrame especificado no parâmetro right deve ser utilizado como chave de junção.

# ADICIONANDO INFORMAÇÕES
---

## Criando as colunas `quartos`, `suites` e `banheiros`

### `str.extractall`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extractall.html)

### `unstack`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.unstack.html)

### `droplevel`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.droplevel.html)

## Criando novas classificações

### Com dados categóricos - `map`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html)

In [ ]:
tipo_uso = {
    'Apartamento': 'Residencial',
    'Casa': 'Residencial',
    'Cobertura': 'Residencial',
    'Consultório': 'Comercial',
    'Imóvel Comercial': 'Comercial',
    'Loja': 'Comercial',
    'Sala Comercial': 'Comercial'
}

#### `insert`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.insert.html)

### Com dados numéricos - `cut`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html)

#### Utilizando classes fixas

#### Utilizando classes personalizadas

## Novas colunas derivadas das informações existentes

### Valor por m²

#### `apply`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html)

### Piscina?

# TABULAÇÕES E SUMARIZAÇÕES
---

## Agrupamentos

### `aggregate`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.aggregate.html)

### `groupby`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html)

### `value_counts`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.value_counts.html)

#### `to_frame`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.to_frame.html)

## Mudando o formato de um *DataFrame*

### `unstack`

Faz a rotação ou o pivoteamento das linhas para as colunas.

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.unstack.html)

### `stack`

Faz a rotação ou o pivoteamento das colunas de um *DataFrame* para as linhas.

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html)

## Criando tabelas dinâmicas

### `pivot`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html)

#### `droplevel`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.droplevel.html)

#### `reset_index`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html)

### `pivot_table`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html)

# TABULAÇÕES E ESTILIZAÇÃO
---

## Tabulações a partir de informações no formato de listas

### `explode`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.explode.html)

#### `where`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.where.html)

## Estilizando um DataFrame

### `style`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.style.html)

#### `format`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.format.html)

#### `bar`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.bar.html)

#### `applymap`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.applymap.html)

#### `highlight_max` e `highlight_min`

[Documentação: `highlight_max`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.highlight_max.html)

[Documentação: `highlight_min`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.highlight_min.html)

#### `background_gradient`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.background_gradient.html)

#### `to_excel`

[Documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.to_excel.html)

# ANEXOS
---

### Opções para o parâmetro `cmap`

[Documentação](https://matplotlib.org/3.3.2/tutorials/colors/colormaps.html)

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import OrderedDict

cmaps = OrderedDict()

In [ ]:
cmaps['Perceptually Uniform Sequential'] = [
            'viridis', 'plasma', 'inferno', 'magma', 'cividis']

cmaps['Sequential'] = [
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']

cmaps['Sequential (2)'] = [
            'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
            'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
            'hot', 'afmhot', 'gist_heat', 'copper']

cmaps['Diverging'] = [
            'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
            'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic']

cmaps['Cyclic'] = ['twilight', 'twilight_shifted', 'hsv']

cmaps['Qualitative'] = ['Pastel1', 'Pastel2', 'Paired', 'Accent',
                        'Dark2', 'Set1', 'Set2', 'Set3',
                        'tab10', 'tab20', 'tab20b', 'tab20c']

cmaps['Miscellaneous'] = [
            'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
            'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
            'gist_rainbow', 'rainbow', 'jet', 'nipy_spectral',
            'gist_ncar']

In [ ]:
nrows = max(len(cmap_list) for cmap_category, cmap_list in cmaps.items())
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))


def plot_color_gradients(cmap_category, cmap_list, nrows):
    fig, axes = plt.subplots(nrows=nrows)
    fig.subplots_adjust(top=0.95, bottom=0.01, left=0.2, right=0.99)
    axes[0].set_title(cmap_category + ' colormaps', fontsize=14)

    for ax, name in zip(axes, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=plt.get_cmap(name))
        pos = list(ax.get_position().bounds)
        x_text = pos[0] - 0.01
        y_text = pos[1] + pos[3]/2.
        fig.text(x_text, y_text, name, va='center', ha='right', fontsize=10)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axes:
        ax.set_axis_off()


for cmap_category, cmap_list in cmaps.items():
    plot_color_gradients(cmap_category, cmap_list, nrows)

plt.show()